#### Incremental Learning- Model Retraining Approach

In [19]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])

In [20]:
messages.shape

(5572, 2)

In [21]:
from sklearn.model_selection import train_test_split
message_train,message_test=train_test_split(messages)

In [22]:
message_train

,label,message
535,ham,I've not called you in a while. This is hoping...
1326,ham,Yeah jay's sort of a fucking retard
189,ham,Have you got Xmas radio times. If not i will g...
728,ham,"Alright omw, gotta change my order to a half8th"
3924,ham,Okay lor... Will they still let us go a not ah...
...,...,...
3505,ham,Will you be here for food
690,spam,<Forwarded from 448712404000>Please CALL 08712...
677,ham,Maybe?! Say hi to and find out if got his ca...
2348,ham,But i dint slept in afternoon.


In [23]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test=message_test.to_records(index=False)

In [24]:
messages_train

rec.array([('ham', "I've not called you in a while. This is hoping it was l8r malaria and that you know that we miss you guys. I miss Bani big, so pls give her my love especially. Have a great day."),
           ('ham', "Yeah jay's sort of a fucking retard"),
           ('ham', 'Have you got Xmas radio times. If not i will get it now'),
           ...,
           ('ham', 'Maybe?! Say hi to  and find out if  got his card. Great escape or wetherspoons? '),
           ('ham', 'But i dint slept in afternoon.'),
           ('ham', 'So now my dad is gonna call after he gets out of work and ask all these crazy questions.')],
          dtype=[('label', 'O'), ('message', 'O')])

In [25]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [26]:
from creme import metrics
metric=metrics.Accuracy()
# Training the model row by row
for label,sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)  

In [27]:
### Training Data Accuracy
metric

Accuracy: 95.38%

In [28]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messages_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

In [29]:
### test Metric
test_metric

Accuracy: 95.66%

In [30]:
model.fit_one("This guy is neutral", "ham")
model.fit_one("Everybody is neutral ", "ham")

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x7efd71b5f5e0>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)